## Quick Start

### Demo 1: Run a WebAssembly function with WasmEdge Rust APIs

As you may know, several mainstream programming languages, such as C/C++, Rust, Go and Python, support to compile their programs into WebAssembly binary. In this demo, we'll introduce how to use WasmEdge Rust APIs to call a WebAssembly function which could be coded in any programming lange mentioned above. 

We use `fibonacci.wasm` in this demo, and the contents of the WebAssembly file are presented as below. The statement, `(export "fib" (func $fib))`, declares an exported function named `fib`. This function computes a fibonacci number with a given `i32` number as input. We'll use the function name later to achieve the goal of computing a fibonacci number.

```wasm
(module
 (export "fib" (func $fib))
 (func $fib (param $n i32) (result i32)
  (if
   (i32.lt_s
    (get_local $n)
    (i32.const 2)
   )
   (return
    (i32.const 1)
   )
  )
  (return
   (i32.add
    (call $fib
     (i32.sub
      (get_local $n)
      (i32.const 2)
     )
    )
    (call $fib
     (i32.sub
      (get_local $n)
      (i32.const 1)
     )
    )
   )
  )
 )
)
```

#### Step 0: Preparation

To use the wasmedge-sys crate in the evcxr_jupyter notebook, we use the following code to specify the path to the crate. The importing process costs 10 or more seconds approximately to compile the wasmedge-sys crate.

In [ ]:
:dep wasmedge-sys = { path = "../../wasmedge-sys" }

println!("*** Step 0 done!")

#### Step 1: Create a WasmEdge AST Module.

In this step,  we'll create a WasmEdge `AST Module` instance from a WebAssembly file. 

- First, create a `Loader` context; 
- Then, load a specified WebAssebly file ("fibonacci.wasm") via the `from_file` method of the `Loader` context. If the process is successful, then a WasmEdge `AST Module` instance is returned.

In [ ]:
use wasmedge_sys::Loader;
use std::path::PathBuf;

// create a Loader context
let loader = Loader::create(None).expect("fail to create a Loader context");

// load a wasm module from a specified wasm file, and return a WasmEdge AST Module instance
let path = PathBuf::from("fibonacci.wasm");
let mut module = loader.from_file(path).expect("fail to load the WebAssembly file");

println!("*** Step 1 done!")

#### Step 2: Create a WasmEdge `Vm` context.

In WasmEdge, a `Vm` defines an running environment, in which all various of instances and contexts are stored and mantained. In the demo code below, we explicitly create a WasmEdge `Store` context, and then use it as one of inputs in the creation of a `Vm` context. If not specify a `Store` context explicily, then `Vm` will create a store by itself.

In [ ]:
use wasmedge_sys::{Config, Store, Vm};

// create a Config context
let result = Config::create();
assert!(result.is_ok());
let config = result.unwrap();

// create a Store context
let result = Store::create();
assert!(result.is_ok(), "Failed to create Store instance");
let store = result.unwrap();

// create a Vm context with the given Config and Store
let result = Vm::create(Some(&config), Some(&store));
assert!(result.is_ok());
let vm = result.unwrap();

println!("*** Step 2 done!")

#### Step 3: Invoke the `fib` function.

In [Step 1], we got a module which hosts the target `fib` function defined in the WebAssembly. Now, we can call the function via the `run_wasm_from_module` method of the `Vm` context by passing the exported function name, `fib`.

In [ ]:
use wasmedge_sys::Value;

// run a function
let result = vm.run_wasm_from_module(&mut module, "fib", [Value::from_i32(5)]);
assert!(result.is_ok(), "Failed to run the target function in the module");
let returns = result.unwrap();
println!("The result of fib(5) is {}", returns[0].to_i32());

println!("*** Step 3 done!")